In [1]:
import pandas as pd
import numpy as np
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None

In [32]:
df_circuits = pd.read_csv('01_raw/circuits.csv')

df_cons_res = pd.read_csv('01_raw/constructor_results.csv')

df_cons_stand = pd.read_csv('01_raw/constructor_standings.csv')

df_cons = pd.read_csv('01_raw/constructors.csv')

df_dri_stand = pd.read_csv('01_raw/driver_standings.csv')

df_driver = pd.read_csv('01_raw/drivers.csv')

df_laptimes = pd.read_csv('01_raw/lap_times.csv')

df_pits = pd.read_csv('01_raw/pit_stops.csv')

df_quali = pd.read_csv('01_raw/qualifying.csv')

df_races = pd.read_csv('01_raw/races.csv')

df_res = pd.read_csv('01_raw/results.csv')

df_seasons = pd.read_csv('01_raw/seasons.csv')

df_sprintres = pd.read_csv('01_raw/sprint_results.csv')

df_status = pd.read_csv('01_raw/status.csv')

In [3]:
df_laptimes.shape

(515715, 6)

In [4]:
df_sprintres.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,fastestLapTime,statusId
0,1,1061,830,9,33,2,1,1,1,3,17,25:38.426,1538426,14,1:30.013,1
1,2,1061,1,131,44,1,2,2,2,2,17,+1.430,1539856,17,1:29.937,1
2,3,1061,822,131,77,3,3,3,3,1,17,+7.502,1545928,17,1:29.958,1
3,4,1061,844,6,16,4,4,4,4,0,17,+11.278,1549704,16,1:30.163,1
4,5,1061,846,1,4,6,5,5,5,0,17,+24.111,1562537,16,1:30.566,1


In [11]:
df_pits.raceId.max()

1074

In [6]:
df_quali.head()

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236


In [14]:
df_races.head()

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,29/3/2009,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,5/4/2009,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,19/4/2009,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,26/4/2009,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,10/5/2009,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...


In [20]:
df_races[df_races['year']==1950]

,raceId,year,round,circuitId,name,date,time,url
832,833,1950,1,9,British Grand Prix,13/5/1950,\N,http://en.wikipedia.org/wiki/1950_British_Gran...
833,834,1950,2,6,Monaco Grand Prix,21/5/1950,\N,http://en.wikipedia.org/wiki/1950_Monaco_Grand...
834,835,1950,3,19,Indianapolis 500,30/5/1950,\N,http://en.wikipedia.org/wiki/1950_Indianapolis...
835,836,1950,4,66,Swiss Grand Prix,4/6/1950,\N,http://en.wikipedia.org/wiki/1950_Swiss_Grand_...
836,837,1950,5,13,Belgian Grand Prix,18/6/1950,\N,http://en.wikipedia.org/wiki/1950_Belgian_Gran...
837,838,1950,6,55,French Grand Prix,2/7/1950,\N,http://en.wikipedia.org/wiki/1950_French_Grand...
838,839,1950,7,14,Italian Grand Prix,3/9/1950,\N,http://en.wikipedia.org/wiki/1950_Italian_Gran...


In [12]:
df_sprintres[df_sprintres['raceId'] == '990']

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,fastestLapTime,statusId


In [15]:
df_sprintres.shape

(60, 16)

In [22]:
df_cons_res.head()

,constructorResultsId,raceId,constructorId,points,status
0,1,18,1,14.000,\N
1,2,18,2,8.000,\N
2,3,18,3,9.000,\N
3,4,18,4,5.000,\N
4,5,18,5,2.000,\N


In [23]:
df_cons_stand.head()

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins
0,1,18,1,14.000,1,1,1
1,2,18,2,8.000,3,3,0
2,3,18,3,9.000,2,2,0
3,4,18,4,5.000,4,4,0
4,5,18,5,2.000,5,5,0


In [24]:
df_cons.head()

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso


In [25]:
df_cons.nationality.unique()

array(['British', 'German', 'French', 'Italian', 'Japanese', 'Austrian',
       'Indian', 'Dutch', 'Russian', 'Swiss', 'Irish', 'Hong Kong',
       'Brazilian', 'Canadian', 'Mexican', 'American', 'Australian',
       'New Zealand', 'South African', 'Rhodesian', 'Belgium',
       'East German', 'Spanish', 'Malaysian'], dtype=object)

In [27]:
df_driver.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [28]:
df_dri_stand.head()

,driverStandingsId,raceId,driverId,points,position,positionText,wins
0,1,18,1,10.000,1,1,1
1,2,18,2,8.000,2,2,0
2,3,18,3,6.000,3,3,0
3,4,18,4,5.000,4,4,0
4,5,18,5,4.000,5,5,0


In [33]:
df_status.head()

,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine
